In [104]:
!pip3 install plotly-express
!pip3 install plotly==5.22.0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [39]:
import duckdb
%load_ext sql

Deploy Shiny apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [40]:
con = duckdb.connect('main.db')
%sql con --alias duckdb


In [41]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [37]:
con.execute('INSTALL spatial;')
con.execute('LOAD spatial;')


In [35]:
weather = con.execute("""CREATE TABLE weather AS SELECT * FROM read_csv('s3://noaa-gsod-pds/2024/*.csv', auto_detect=true);""")


In [44]:
%%sql
ALTER TABLE weather ADD COLUMN station_coordinates geometry;



,Success


In [46]:
%%sql
UPDATE weather SET station_coordinates = ST_Point(latitude,longitude)

,Count
0,1470895


In [47]:
con.execute("""SELECT * FROM weather LIMIT 10""").df()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,station_coordinates
0,01001099999,2024-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.6,24,20.5,24,...,27.8,32.9,,17.4,,0.01,G,999.9,000000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,01001099999,2024-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",33.3,24,30.9,24,...,20.6,34.5,,31.1,,99.99,,999.9,111000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,01001099999,2024-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",29.7,24,27.2,24,...,30.1,34.3,,25.3,,0.00,G,999.9,000000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
3,01001099999,2024-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",19.5,24,14.8,24,...,32.1,28.8,,14.7,,0.00,G,999.9,001000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
4,01001099999,2024-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",20.6,24,16.5,24,...,31.9,26.6,*,16.0,,0.06,G,999.9,001000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
5,01001099999,2024-01-06,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",20.6,24,17.0,24,...,25.4,22.8,,16.0,,0.03,G,999.9,001000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
6,01001099999,2024-01-07,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",27.0,24,21.7,24,...,24.7,34.7,,17.1,,0.01,G,999.9,011000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
7,01001099999,2024-01-08,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",40.1,24,35.4,24,...,38.1,45.5,,32.9,,0.05,G,999.9,010000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
8,01001099999,2024-01-09,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",34.6,24,31.5,24,...,37.1,42.1,,29.7,,0.00,G,999.9,010000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
9,01001099999,2024-01-10,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",26.0,24,23.9,24,...,37.1,41.9,,20.3,,0.01,G,999.9,010000,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."


In [48]:
import requests

In [91]:
df_list = []

url_list = ["https://api.census.gov/data/2020/dec/dhc?get=NAME&for=county:*&in=state:*",
                "https://api.census.gov/data/2020/dec/dhc?get=P1_001N&for=county:*&in=state:*", ###TOTAL POPULATION	
                "https://api.census.gov/data/2020/dec/dhc?get=P10_001N&for=county:*&in=state:*", ###RACE FOR THE POPULATION 18 YEARS AND OVER		
                "https://api.census.gov/data/2020/dec/dhc?get=P10_003N&for=county:*&in=state:*", ###White alone
                "https://api.census.gov/data/2020/dec/dhc?get=P10_004N&for=county:*&in=state:*", ###Black or African American alone
                "https://api.census.gov/data/2020/dec/dhc?get=P10_005N&for=county:*&in=state:*", ###American Indian and Alaska Native alone
                "https://api.census.gov/data/2020/dec/dhc?get=P10_006N&for=county:*&in=state:*", ###Asian alone
                "https://api.census.gov/data/2020/dec/dhc?get=P10_007N&for=county:*&in=state:*", ###American Indian and Alaska Native alone
                "https://api.census.gov/data/2020/dec/dhc?get=P10_008N&for=county:*&in=state:*", ###American Indian and Alaska Native alone
                "https://api.census.gov/data/2020/dec/dhc?get=P10_009N&for=county:*&in=state:*", ###!!Total:!!Population of two or more races:
                "https://api.census.gov/data/2020/dec/dhc?get=P11_002N&for=county:*&in=state:*", ###!!Hispanic or Latino	
                "https://api.census.gov/data/2020/dec/dhc?get=P11_003N&for=county:*&in=state:*" ###!!Not Hispanic or Latino:
]

def generate_tbl(url: str):
    response = requests.get(url).json()
    response_df = pd.DataFrame(response[1:],columns=response[0])
    return response_df
    
        



In [92]:
for url in url_list:
    df_list.append(generate_tbl(url))

In [93]:
df_list

[                                NAME state county
 0          Madera County, California    06    039
 1           Marin County, California    06    041
 2        Mariposa County, California    06    043
 3       Mendocino County, California    06    045
 4          Merced County, California    06    047
 ...                              ...   ...    ...
 3216         Kern County, California    06    029
 3217        Kings County, California    06    031
 3218         Lake County, California    06    033
 3219       Lassen County, California    06    035
 3220  Los Angeles County, California    06    037
 
 [3221 rows x 3 columns],
        P1_001N state county
 0       156255    06    039
 1       262321    06    041
 2        17131    06    043
 3        91601    06    045
 4       281202    06    047
 ...        ...   ...    ...
 3216    909235    06    029
 3217    152486    06    031
 3218     68163    06    033
 3219     32730    06    035
 3220  10014009    06    037
 
 [3221 row

In [105]:
from functools import reduce
df = reduce(lambda x, y: pd.merge(x, y, on = ['state','county']), df_list)

In [107]:
df['fips'] = df['state'] + df['county']

In [109]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [112]:
import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='P1_001N',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'P1_001N':'TOTAL POPULATION'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()